# PyPitch: End-to-End Pipeline & Feature Implementation

This notebook demonstrates the step-by-step implementation, integration, testing, and usage of the PyPitch pipeline. Each feature is documented, tested, and ready for review and feedback.

---

## 1. Feature Implementation: Step-by-Step Development

We will implement each feature one by one, ensuring modularity, clarity, and documentation. Review and feedback are welcome after each step.

### 1.1 Zero-Config CricsheetLoader and Adapters

We start by implementing and demonstrating the zero-config CricsheetLoader, which allows users to load and explore Cricsheet data out-of-the-box.

In [10]:
# Force use of local source code
import sys, os

# Clear any cached pypitch module
if 'pypitch' in sys.modules:
    del sys.modules['pypitch']

# Add local project root to sys.path at the beginning
project_root = 'd:\\Srijan\\PyPitch'
sys.path.insert(0, project_root)

# Now import from local source
from pypitch.sources import CricsheetLoader

# Initialize loader with correct data path
loader = CricsheetLoader(data_dir='d:\\Srijan\\PyPitch\\data\\raw\\ipl')

# List available match IDs
match_ids = loader.get_match_ids()
print(f"Found {len(match_ids)} matches. Example IDs: {match_ids[:3]}")

# Load data for the first match
if match_ids:
    match_data = loader.get_match_data(match_ids[0])
    print(f"Loaded match: {match_data['match_id']}")
    print(f"Venue: {match_data['info'].get('venue')}")
    print(f"Number of innings: {len(match_data['events'])}")
else:
    print("No matches found in the data directory.")

Found 1169 matches. Example IDs: ['1082591', '1082592', '1082593']
Loaded match: 1082591
Venue: Rajiv Gandhi International Stadium, Uppal
Number of innings: 2


### 1.2 Debug Mode for Eager Execution

Debug mode forces all queries to execute eagerly, surfacing errors immediately instead of lazily. This is essential for development and debugging.

In [13]:
# Force use of local source code
import sys, os

# Clear any cached pypitch module
if 'pypitch' in sys.modules:
    del sys.modules['pypitch']

# Add local project root to sys.path at the beginning
project_root = 'd:\\Srijan\\PyPitch'
sys.path.insert(0, project_root)

# Enable debug mode for eager execution
import pypitch
pypitch.set_debug_mode(True)
print("Debug mode enabled: Queries will execute eagerly to surface errors immediately.")

# Example: In debug mode, any query errors will be raised right away
# (Actual query execution would depend on having a full session setup)

# Disable debug mode for production
pypitch.set_debug_mode(False)
print("Debug mode disabled: Queries use lazy execution for performance.")

Debug mode enabled: Queries will execute eagerly to surface errors immediately.
Debug mode disabled: Queries use lazy execution for performance.


### 1.3 Baseline Win Probability Model

PyPitch ships with a default, pre-trained win probability model for T20 cricket. No setup required - just use it out-of-the-box.

In [15]:
# Force use of local source code
import sys, os
if 'pypitch' in sys.modules:
    del sys.modules['pypitch']
project_root = 'd:\\Srijan\\PyPitch'
sys.path.insert(0, project_root)

# Use the baseline win probability model
import pypitch

# Example: Team chasing 150, scored 50 in 10 overs with 2 wickets down
result = pypitch.win_probability(target=150, current_runs=50, wickets_down=2, overs_done=10.0)
print(f"Win probability: {result['win_prob']:.2%}")

# Use the WinPredictor class directly
model = pypitch.WinPredictor()
prob = model.predict(target=150, current_runs=50, wickets_down=2, overs_done=10.0)
print(f"Direct model prediction: {prob:.2%}")

# Swap with a custom model (example)
custom_model = pypitch.WinPredictor({"intercept": 0.5, "runs_remaining": -0.02, "balls_remaining": 0.01,
                                     "wickets_remaining": 0.2, "run_rate_required": -0.25, "run_rate_current": 0.2})
pypitch.set_win_model(custom_model)
result_custom = pypitch.win_probability(target=150, current_runs=50, wickets_down=2, overs_done=10.0)
print(f"Custom model prediction: {result_custom['win_prob']:.2%}")

Win probability: 47.75%
Direct model prediction: 47.75%
Custom model prediction: 31.00%


### 1.4 Impact Player Rule Support

PyPitch supports flexible team configurations, including the Impact Player rule that allows 12 players per team. No code changes required - just configure and go.

In [17]:
# Force use of local source code
import sys, os
if 'pypitch' in sys.modules:
    del sys.modules['pypitch']
project_root = 'd:\\Srijan\\PyPitch'
sys.path.insert(0, project_root)

# Demonstrate Impact Player configuration
import pypitch

# Standard T20 configuration (11 players)
standard_config = pypitch.MatchConfig.t20()
print(f"Standard T20: {standard_config.max_players_per_team} players per team")
print(f"Total overs: {standard_config.total_overs}, Balls per over: {standard_config.balls_per_over}")

# Impact Player T20 configuration (12 players)
impact_config = pypitch.MatchConfig.t20_impact_player()
print(f"Impact Player T20: {impact_config.max_players_per_team} players per team")
print(f"Total overs: {impact_config.total_overs}, Balls per over: {impact_config.balls_per_over}")

# Custom configuration for any format
custom_config = pypitch.MatchConfig(
    total_overs=20, 
    balls_per_over=6, 
    max_players_per_team=13,  # Even more flexible
    powerplay_overs=6
)
print(f"Custom format: {custom_config.max_players_per_team} players per team")
print(f"Total balls: {custom_config.total_balls}")

print("\n✅ PyPitch is ready for Impact Player and any future rule changes!")

Standard T20: 11 players per team
Total overs: 20, Balls per over: 6
Impact Player T20: 12 players per team
Total overs: 20, Balls per over: 6
Custom format: 13 players per team
Total balls: 120

✅ PyPitch is ready for Impact Player and any future rule changes!
